In [78]:
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Embedding
from keras.models import Sequential
from keras.layers import Flatten, Dense, LSTM

from keras.layers import Embedding
import pandas as pd
import numpy as np

In [58]:
df = pd.read_csv('/Users/babyhandzzz/Desktop/ELEPH@NT/Datasets/clean_IMDB.csv',usecols=['text','label'])
glove_path = '/Users/babyhandzzz/Desktop/ELEPH@NT/Datasets'

In [59]:
# define tokenization method
tokenizer = Tokenizer(nb_words=500)
# fit tokenizer on the data
tokenizer.fit_on_texts(df['text'])
# generate sequences
sequences = tokenizer.texts_to_sequences(df['text'])
# unique number to word indexation
word_index = tokenizer.word_index

In [61]:
# padding all the vectors to the same lenght
data = pad_sequences(sequences, maxlen=300)

In [62]:
# OHE for data labels
labels = to_categorical(np.asarray(df['label']))

In [63]:
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (50000, 300)
Shape of label tensor: (50000, 2)


In [64]:
# Train Test Split
indices = np.arange(data.shape[0]) 
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(0.3 * data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_val = data[-nb_validation_samples:]
y_val = labels[-nb_validation_samples:]

In [65]:
# Dictionary that maps integer indexes to the correposnding vectors.
embeddings_index = {}
                                    # don't forget that glove file is a .txt file
f = open(os.path.join(glove_path, 'glove.6B.50d.txt'))
for line in f:
    # split every value on space
    values = line.split()
    # the word itself is the first item in every line
    word = values[0]
    # the actual vector is the vector that follows the word
    coefs = np.asarray(values[1:], dtype='float32')
    # dictionary mapping of the words to their vectors
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [66]:
# np.array matrix that 
embedding_matrix = np.zeros((len(word_index) + 1, 50))
                # word index is a dictionary that contains tokenized words from the dataset
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# By doing all of this stuff we obtain a matrix with all the vectors that represent words from the vocabulary.        

In [82]:
model = Sequential()
model.add(Embedding(1000, 32))
model.add(LSTM(32))
model.add(Dense(2, activation = "sigmoid"))

model.compile(optimizer = "adam",
             loss = "binary_crossentropy",
             metrics = ["acc"])
#history = model.fit(x_train, y_train, epochs=10, batch_size=128)